## Використання машинного навчання в Python для прогнозування погоди
### Збір даних про погоду з WeatherUnderground

### Вступ

Це перша стаття багатостороння серія з використанням Python та Machine Learning для побудови моделей для прогнозування на основі даних, отриманих з Weather Underground. Серія буде складатися з трьох різних статей, що описують основні аспекти проекту Machine Learning. Теми, що підлягають обговоренню:

  1.  збір та обробка даних
  2.  моделі лінійної регресії
  3.  моделі нейронних мереж

Дані, використані в цій серії, будуть зібрані за допомогою безкоштовного API сервысу Weather Underground. Я буду використовувати бібліотеку запитів, щоб взаємодіяти з API, щоб отримувати дані про погоду з 2015 року для міста Лінкольн-Небраска. Після збирання ці дані потрібно буде обробляти та об'єднати у формат, який підходить для аналізу даних, і потім очистити. Ця стаття завершиться обговоренням тестування та перевірки моделей лінійної регресії.

Остання стаття буде зосереджена на використанні нейронних мереж. Я буду порівнювати процес побудови моделі нейронної мережі, інтерпретувати результати та загальну точність між моделлю лінійної регресії, побудованої в попередній статті та моделі нейронної мережі.

### Ознайомлення з Weather Underground API

Weather Underground це компанія, яка збирає та поширює дані про різні метеорологічні вимірювання по всьому світу.
Компанія надає широкий спектр API, доступних як для комерційного, так і для некомерційного використання.
У цій статті я розповім про те, як програмно отримати щоденні дані про погоду від Weather Underground, використовуючи їъ безкоштовний API який доступний для некомерційних цілей.
Якщо ви хочете слідувати за туторіалом, ви маєте зареєструвати безкоштовний обліковий запис розробника тут [тут](https://www.wunderground.com/weather/api/).
Цей обліковий запис надає ключ API для доступу до веб-служби зі швидкістю 10 запитів за хвилину і до 500 запитів за день.
Weather Underground надає безліч різних API веб-служб для доступу до даних, але нам знадобиться лиш API історії.
API історії надає звіт про різні метеорологічні виміри для міста та штату в певний день.
Формат запиту для ресурсу API історії виглядає наступним чином:

http://api.wunderground.com/api/API_KEY/history_YYYYMMDD/q/STATE/CITY.json
  -  API_KEY: це API_KEY, що Weather Underground надає вашому обліковому запиту
  -  YYYYMMDD: це рядок, що представляє цільову дату вашого запиту
  -  STATE: це дволітерна державна абревіатура в США
  -  CITY: це назва міста, пов'язаного із запитаною вами державою

### Створення запитів до API

Щоб надсилати запити до History API Weather Underground і обробляти повернуті дані, я скористаюсь декількома стандартними бібліотеками, а також деякими популярними сторонніми бібліотеками.
Нижче наведено таблицю бібліотек, які я буду використовувати, та їх опис.
Для інструкцій з установки, будь ласка, зверніться до переліченої документації.


| Быбліотека     | Опис використання                        | Джерело           |
|-------------------------------------------------------------------------------------|
| [datetime](https://docs.python.org/3/library/datetime.html)    | Використовується для інкрементування дня для запитів за день              | Стандартна бібліотека |
| [time](https://docs.python.org/3/library/time.html)        |Використовується для затримки запитів до 10 за хвилину | Стандартна бібліотека |
| [collections](https://docs.python.org/3/library/collections.html#collections.namedtuple) |Використовуйте namedtuples для стукртурування колекції даних  | Стандартна бібліотека |
| [pandas](http://pandas.pydata.org/)      | Використовується для обробки, систематизації та очищення даних      | стороння бібліотека    |
| [requests](http://docs.python-requests.org/en/master/)    | Використовується для створення мережевих запитів до API        | стороння бібліотека  |
| [matplotlib](https://matplotlib.org/) | Використовується для візуального аналізу | стороння бібліотека |

Давайте почнемо з імпорту цих бібліотек.

In [6]:
from datetime import datetime, timedelta
import time
from collections import namedtuple
import pandas as pd
import requests
import matplotlib.pyplot as plt

Тепер я визначу пару констант, що являють собою мій API_KEY, і BASE_URL кінцевої точки API, яку я запитую.
Зверніть увагу, що вам потрібно зареєструвати обліковий запис у Weather Underground і отримати власний API_KEY.
До моменту публікації цієї статті я відключу свій API_KEY.

BASE_URL - це рядок з двома заповнювачами, представленими фігурними дужками.
Перші {} будуть заповнені API_KEY, а другі {} будуть замінені датою.
Обидва значення будуть інтерпольовані в рядку BASE_URL за допомогою [str.format(...)](https://docs.python.org/2/library/stdtypes.html#str.format) function.

In [7]:
API_KEY = '7052ad35e3c73564'
BASE_URL = "http://api.wunderground.com/api/{}/history_{}/q/NE/Lincoln.json"

Далі я ініціалізую цільову дату - перший день 2015 року.
Тоді я вкажу властивості, які я хотів би проаналізувати з відповідей, отриманих від API.
Властивості - це просто ключі, що містяться в історії -> dailysummary частина повної відповіді JSON.
Ці властивості використовуються для визначення namedtuple під назвою DailySummary, який я буду використовувати для упорядкування даних окремих запитів у списку елементами якого є tuples DailySummary.

In [8]:
target_date = datetime(2016, 5, 16)
features = ["date", "meantempm", "meandewptm", "meanpressurem", "maxhumidity", "minhumidity", "maxtempm",
            "mintempm", "maxdewptm", "mindewptm", "maxpressurem", "minpressurem", "precipm"]
DailySummary = namedtuple("DailySummary", features)

У цьому розділі я буду робити запити до API та збирати успішні відповіді за допомогою функції, визначеної нижче.
Ця функція приймає параметри url, api_key, target_date та days.

In [9]:
def extract_weather_data(url, api_key, target_date, days):
    records = []
    for _ in range(days):
        request = BASE_URL.format(API_KEY, target_date.strftime('%Y%m%d'))
        response = requests.get(request)
        if response.status_code == 200:
            data = response.json()['history']['dailysummary'][0]
            records.append(DailySummary(
                date=target_date,
                meantempm=data['meantempm'],
                meandewptm=data['meandewptm'],
                meanpressurem=data['meanpressurem'],
                maxhumidity=data['maxhumidity'],
                minhumidity=data['minhumidity'],
                maxtempm=data['maxtempm'],
                mintempm=data['mintempm'],
                maxdewptm=data['maxdewptm'],
                mindewptm=data['mindewptm'],
                maxpressurem=data['maxpressurem'],
                minpressurem=data['minpressurem'],
                precipm=data['precipm']))
        time.sleep(6)
        target_date += timedelta(days=1)
    return records

Я починаю з визначення списку records, який містить DailySummary namedtuples.
Цикл ітерується по числу днів які передані в функцію з параметром days.

Потім запит відформатовано за допомогою функції str.format () для інтерполяції API_KEY та target_date.
Після форматування змінна запиту передається методу бібліотеки requests get (), а відповідь присвоюється змінній response.


Отримавши відповідь, я хочу переконатися, що запит був успішним, перевіривши, що код стану рівний 200.
Якщо він буде успішним, я проаналізую тіло відповіді в JSON за допомогою методу json () повернутого response object.
Заклик методу, пов'язаний з тим же самим методом json () Я вибираю індекси історії та щоденних структур підсумків, потім схоплюю перший елемент у списку dailysummary та призначаю його змінній data.

Тепер, коли у мене є структура даних типу словарь, на яку посилається змінна data, я можу вибрати потрібні поля та інформувати новий екземпляр DailySummary namedtuple, який додається до списку записів.

### Перша партія запитів

Без подальшої затримки я почну з першого набору запитів на максимальну кількість виділених запитів для безкоштовного акаунту розробника - 500.
Я пропоную взяти з собою каву  (або інший напій) та подивитися ваше улюблене телевізійне шоу, оскільки ця функція буде виконуватися принаймні годину в залежності від швикості мережі, ми вичерпаємо кількість запитів, і це тільки близько половини даних, з якими ми будемо працювати.
Отже, повернімося завтра, де я завершу останню партію запитів, після чого ми можемо почати працювати над обробкою та форматуванням даних у спосіб, придатний для нашого Machine Learning проекту .

In [10]:
records = extract_weather_data(BASE_URL, API_KEY, target_date, 500)

### Завершення виведення даних

Гаразд, тепер, це новий день, ми маємо чистий список і до 500 запитів, які можна зробити до API Weather History Underground.
Наша партія з 500 запитів, виданих вчора, розпочалася 1 січня 2015 року і закінчилася 15 травня 2016 року (якщо ви не отримали жодних невдалих запитів).
Ще раз дозвольте нам почати ще одну партію з 500 запитів, але не залишайте мене на день, тому що як тільки цей останній пакет даних буде зібраний, ми збираємося почати його форматування у Pandas DataFrame і отримати потенційно корисні данні.

In [ ]:
# if you closed our terminal or Jupyter Notebook reinitialize your imports and
# variables first and remember to set your target_date to datetime(2016, 5, 16)
records += extract_weather_data(BASE_URL, API_KEY, target_date, 500)

### Налаштування нашого Pandas DataFrame

Тепер, коли у мене є хороший і значний список записів DailySummary іменованих кортежів, я буду використовувати його для створення Pandas DataFrame.
Pandas DataFrame - це дуже корисна структура даних для багатьох завдань програмування, які найбільш популярні для чищення та обробки даних, які використовуються в проектах машинного навчання (або експериментах).
Я буду використовувати конструктор класу `Pandas.DataFrame (...)` для інстанції об'єкта DataFrame.
Параметри, передані конструктору, - це записи, які представляють дані для DataFrame, список властивостей, який я також використовував для визначення іменованого кортежу DailySummary, який буде вказувати на стовпці DataFrame.
Метод set_index () прикріплений до екземпляру DataFrame, щоб вказати дату як індекс.

In [ ]:
df = pd.DataFrame(records, columns=features).set_index('date')

### Визначення властивостей

Проекти машинного навчання, також називаються експериментами, часто мають кілька характеристик, які трохи оксимононічні.
Під цим я маю на увазі, що цілком корисно мати предметні знання в досліджуваній області, щоб допомогти у виборі значущих властивостей для дослідження в парі з вдумливим припущенням про можливі патерни даних.

Проте я також бачив дуже впливові пояснювальні змінні, і модель виникає з майже наївних або принаймні дуже відкритих і мінімальних припущень щодо даних.
Маючи інтуїцію яка базується на знаннях, щоб знати, де шукати потенційно корисні властивості та патерни, а також можливість неузгоджено шукати непередбачувані характеристики непредвзято, це надзвичайно важлива частина успішного аналітичного проекту.

У зв'язку з цим ми виділили кілька властивостей під час аналізу отриманих щоденних підсумкових даних, які будуть використовуватися в нашому дослідженні.
Однак я повністю очікую, що багато хто з них виявиться або неінформативноми для прогнозування погодних умов або невідповідны кандидати що залежать від типу використовуваної моделі, але ключовим є те, що ви просто не знаєте, поки не ретельно вивчите дані.

Зараз я не можу сказати, що я маю значні знання в метеорології або прогнозуванні погоди, але я зробив мінімальний пошук підготовчої роботи з використання машинного навчання для прогнозування погоди.
Як виявляється, є багато дослідницьких статей на цю тему, і в 2016 році Холмстрем, Лю та Во описують використання лінійної регресії, щоб зробити прогнозування погоди.
У їхній статті [Machine Learning Applied to Weather Forecasting](https://pdfs.semanticscholar.org/2761/8afb77c5081d942640333528943149a66edd.pdf), яка застосовується до прогнозування погоди, вони використовували дані про погоду за попередні два дні для наступних сими днів.

*  max temperature - максимальна температура
*  min temperature - мінімальна температура
*  mean humidity - середня вологість
*  mean atmospheric pressure - середній атмосферний тиск

Я буду розширювати свій список властивостей, використовуючи ті, що перераховані нижче, і замість того, щоб використовувати лише попередні два дні, я буду використовувати три дні.

*  mean temperature - середня температура
*  mean dewpoint - середня точка роси
*  mean pressure - середній тиск 
*  max humidity - максимальна вологість
*  min humidity - мінімальна вологість
*  max dewpoint - максимальна точка роси 
*  min dewpoint - мінімальна точка роси
*  max pressure - максимальний тиск
*  min pressure - мінімальний тиск
*  precipitation - опади

Тож далі слід з'ясувати спосіб включення цих нових властивостей у вигляді стовпців у нашому DataFrame.
Для цього я буду створювати меншу підмножину поточного DataFrame, щоб полегшити роботу з розробкою алгоритму для створення цих властивостей.
Я буду створювати `tmp` DataFrame, що складається всього з 10 записів і властивостей meantempm і meandewptm.

In [12]:
tmp = df[['meantempm', 'meandewptm']].head(10)
tmp

,meantempm,meandewptm
date,,
2015-01-01,-6,-12
2015-01-02,-6,-9
2015-01-03,-4,-11
2015-01-04,-14,-19
2015-01-05,-9,-14
2015-01-06,-10,-15
2015-01-07,-16,-22
2015-01-08,-7,-12
2015-01-09,-11,-19


Давайте розіб'ємо те, що ми сподіваємося виконати, а потім переведемо його в код.
Для кожного дня (рядка) і для заданої властивості (стовпця) я хотів би знайти значення для цієї функції за N днів до.
Для кожного значення N (1-3 у нашому випадку) я хочу створити новий стовпчик для цієї властивості, що відповідає  N-ному виміру попереднього дня.

In [13]:
# 1 day prior
N = 1

# target measurement of mean temperature
feature = 'meantempm'

# total number of rows
rows = tmp.shape[0]

# a list representing Nth prior measurements of feature
# notice that the front of the list needs to be padded with N
# None values to maintain the constistent rows length for each N
nth_prior_measurements = [None]*N + [tmp[feature][i-N] for i in range(N, rows)]

# make a new column name of feature_N and add to DataFrame
col_name = "{}_{}".format(feature, N)
tmp[col_name] = nth_prior_measurements
tmp

,meantempm,meandewptm,meantempm_1
date,,,
2015-01-01,-6,-12,None
2015-01-02,-6,-9,-6
2015-01-03,-4,-11,-6
2015-01-04,-14,-19,-4
2015-01-05,-9,-14,-14
2015-01-06,-10,-15,-9
2015-01-07,-16,-22,-10
2015-01-08,-7,-12,-16
2015-01-09,-11,-19,-7


Гаразд, здається, у нас є основні кроки, необхідні для створення наших нових властивостей. 
Зараз я оберну ці кроки в функцію та виконаю її, побудуючи всі потрібні властивості.  

In [ ]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_meassurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_meassurements

Тепер я напишу цикл для перебору над властивостей у спискау властивостей, визначеним раніше, і для кожної властивості, яка не є датою, і за N днів 1 до 3 викличу нашу функцію додати похідні властивості, які ми хочемо оцінити для прогнозування температур.

In [ ]:
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(df, feature, N)

Для перевірки, я подивлюся на стовпці, щоб переконатися, що вони виглядають як очікується.

In [15]:
df.columns

Index(['meantempm', 'meandewptm', 'meanpressurem', 'maxhumidity',
       'minhumidity', 'maxtempm', 'mintempm', 'maxdewptm', 'mindewptm',
       'maxpressurem', 'minpressurem', 'precipm', 'meantempm_1', 'meantempm_2',
       'meantempm_3', 'meandewptm_1', 'meandewptm_2', 'meandewptm_3',
       'meanpressurem_1', 'meanpressurem_2', 'meanpressurem_3',
       'maxhumidity_1', 'maxhumidity_2', 'maxhumidity_3', 'minhumidity_1',
       'minhumidity_2', 'minhumidity_3', 'maxtempm_1', 'maxtempm_2',
       'maxtempm_3', 'mintempm_1', 'mintempm_2', 'mintempm_3', 'maxdewptm_1',
       'maxdewptm_2', 'maxdewptm_3', 'mindewptm_1', 'mindewptm_2',
       'mindewptm_3', 'maxpressurem_1', 'maxpressurem_2', 'maxpressurem_3',
       'minpressurem_1', 'minpressurem_2', 'minpressurem_3', 'precipm_1',
       'precipm_2', 'precipm_3'],
      dtype='object')

Відмінно! Схоже, ми маємо те, що нам потрібно. Наступне, що я хочу зробити - це оцінити якість даних та очистити їх там, де це потрібно.

### Найважливіша частина - Очищення даних
Як зазначає назва розділу, найважливішою частиною аналітичного проекту -  переконатися, що ви використовуєте якісні дані.
Прислів'я  "Сміття на вході, сміття на виході ", як завжди, доречне, коли мова йде про машинне навчання.
Проте очистка данних частину як частина аналітичного проекту, є не просто однією з найважливіших частин, а й найбільш трудомістким.
Щоб забезпечити якість даних для даного проекту, в цьому розділі я буду шукати небажані дані, відсутні значення, узгодженість типів даних та [вииди](https://www.wikiwand.com/en/Outlier), а потім прийняти деякі рішення щодо того як їх обробляти, якщо вони виникнуть.

Перше, що я хочу зробити - це видалити будь-які стовпці DataFrame, яким я не цікавлюся, щоб зменшити обсяг даних, з якими я працюю.
Метою проекту є прогнозування майбутньої температури, яка базується на останніх трьох днях погодних вимірів.
Маючи це на увазі, ми хочемо зберегти мінімальну, максимальну та середню температуру на кожен день та також всі нові похідні змінні, які ми додали в останніх розділах.

In [16]:
# make list of original features without meantempm, mintempm, and maxtempm
to_remove = [feature 
             for feature in features 
             if feature not in ['meantempm', 'mintempm', 'maxtempm']]

# make a list of columns to keep
to_keep = [col for col in df.columns if col not in to_remove]

# select only the columns in to_keep and assign to df
df = df[to_keep]
df.columns

Index(['meantempm', 'maxtempm', 'mintempm', 'meantempm_1', 'meantempm_2',
       'meantempm_3', 'meandewptm_1', 'meandewptm_2', 'meandewptm_3',
       'meanpressurem_1', 'meanpressurem_2', 'meanpressurem_3',
       'maxhumidity_1', 'maxhumidity_2', 'maxhumidity_3', 'minhumidity_1',
       'minhumidity_2', 'minhumidity_3', 'maxtempm_1', 'maxtempm_2',
       'maxtempm_3', 'mintempm_1', 'mintempm_2', 'mintempm_3', 'maxdewptm_1',
       'maxdewptm_2', 'maxdewptm_3', 'mindewptm_1', 'mindewptm_2',
       'mindewptm_3', 'maxpressurem_1', 'maxpressurem_2', 'maxpressurem_3',
       'minpressurem_1', 'minpressurem_2', 'minpressurem_3', 'precipm_1',
       'precipm_2', 'precipm_3'],
      dtype='object')

Наступне, що я хочу зробити, - це використати деякі вбудовані функції Pandas, щоб краще зрозуміти ці дані та потенційно визначити окремі області, на яких потрібно сфокусуватися.
Перша функція - це метод DataFrame `info ()`, який, великий сюрприз ..., надає інформацію про DataFrame.
Нас цікавить стовбець "data type".

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2015-01-01 to 2017-09-27
Data columns (total 39 columns):
meantempm          1000 non-null object
maxtempm           1000 non-null object
mintempm           1000 non-null object
meantempm_1        999 non-null object
meantempm_2        998 non-null object
meantempm_3        997 non-null object
meandewptm_1       999 non-null object
meandewptm_2       998 non-null object
meandewptm_3       997 non-null object
meanpressurem_1    999 non-null object
meanpressurem_2    998 non-null object
meanpressurem_3    997 non-null object
maxhumidity_1      999 non-null object
maxhumidity_2      998 non-null object
maxhumidity_3      997 non-null object
minhumidity_1      999 non-null object
minhumidity_2      998 non-null object
minhumidity_3      997 non-null object
maxtempm_1         999 non-null object
maxtempm_2         998 non-null object
maxtempm_3         997 non-null object
mintempm_1         999 non-null object
mintempm_2    

Зверніть увагу, що тип даних кожного стовпця являє собою тип object.
Нам потрібно перетворити всі данні з стовбчиків на дійсні для числового аналізу, який ми сподіваємося виконати.
Для цього я буду використовувати метод DataFrame `apply ()` для застосування методу Pandas to_numeric до всіх значень DataFrame. Параметр error = 'coerce' замінить будь-які текстові значення на Nans.
Загальноприйнятним є пошук текстових значень в даних дикої природи, які зазвичай походять із збирача даних, де дані відсутні або недійсні.

In [18]:
df = df.apply(pd.to_numeric, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2015-01-01 to 2017-09-27
Data columns (total 39 columns):
meantempm          1000 non-null int64
maxtempm           1000 non-null int64
mintempm           1000 non-null int64
meantempm_1        999 non-null float64
meantempm_2        998 non-null float64
meantempm_3        997 non-null float64
meandewptm_1       999 non-null float64
meandewptm_2       998 non-null float64
meandewptm_3       997 non-null float64
meanpressurem_1    999 non-null float64
meanpressurem_2    998 non-null float64
meanpressurem_3    997 non-null float64
maxhumidity_1      999 non-null float64
maxhumidity_2      998 non-null float64
maxhumidity_3      997 non-null float64
minhumidity_1      999 non-null float64
minhumidity_2      998 non-null float64
minhumidity_3      997 non-null float64
maxtempm_1         999 non-null float64
maxtempm_2         998 non-null float64
maxtempm_3         997 non-null float64
mintempm_1         999 non-null float6

Тепер, коли всі наші дані мають тип даних який я хотів, тепер я подивлюся на деяку загальну статистику властивостей і скористаюся [емпіричним правилом](https://en.wikipedia.org/wiki/Rule_of_thumb) для перевірки наявності екстримальних викидів.
Метод DataFrame `describe ()` створює DataFrame, що містить обчислення, середнього, стандартного відхилення, мінімальне, 25 [процентиль](https://en.wikipedia.org/wiki/Percentile), 50 процентиль (або медіана), 75-й процентиль та максимальне значення.
Це може бути дуже корисною інформацією для оцінки розподілу даних.

Я хотів би додати до цієї інформації, обчисливши інший вихідний стовпчик, який вказує на існування викидів.
Емпіричне правило для виявлення екстремального відступу - це значення, менш ніж 3 міжквартильних діапазонів коливаються нижче 25-го процентиля або 3 [міжквартильних діапазонів](https://en.wikipedia.org/wiki/Interquartile_range) вище 75-го процентиля.
Міжквартильний діапазон - це просто різниця між 75-м процентилем і 25-м процентилеєм.

In [19]:
# Call describe on df and transpose it due to the large number of columns
spread = df.describe().T

# precalculate interquartile range for ease of use in next calculation
IQR = spread['75%'] - spread['25%']

# create an outliers column which is either 3 IQRs below the first quartile or
# 3 IQRs above the third quartile
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max'] > (spread['75%']+3*IQR))

# just display the features containing extreme outliers
spread.ix[spread.outliers,]

,count,mean,std,min,25%,50%,75%,max,outliers
maxhumidity_1,997.0,88.107322,9.280627,47.0,83.0,90.0,93.00,100.00,True
maxhumidity_2,997.0,88.106319,9.280152,47.0,83.0,90.0,93.00,100.00,True
maxhumidity_3,997.0,88.093280,9.276775,47.0,83.0,90.0,93.00,100.00,True
maxpressurem_1,997.0,1019.913741,7.755590,993.0,1015.0,1019.0,1024.00,1055.00,True
maxpressurem_2,997.0,1019.917753,7.757705,993.0,1015.0,1019.0,1024.00,1055.00,True
maxpressurem_3,997.0,1019.927783,7.757805,993.0,1015.0,1019.0,1024.00,1055.00,True
minpressurem_1,997.0,1012.317954,7.885743,956.0,1008.0,1012.0,1017.00,1035.00,True
minpressurem_2,997.0,1012.319960,7.886681,956.0,1008.0,1012.0,1017.00,1035.00,True
minpressurem_3,997.0,1012.326981,7.889511,956.0,1008.0,1012.0,1017.00,1035.00,True
precipm_1,997.0,2.593180,8.428058,0.0,0.0,0.0,0.25,95.76,True
